<a href="https://colab.research.google.com/github/takatakamanbou/ML/blob/2024/ML2024_ex10notebookC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ML ex10notebookC

<img width=72 src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/ML/ML-logo.png"> [この授業のウェブページ](https://www-tlab.math.ryukoku.ac.jp/wiki/?ML)


----
## 過適合の抑制とモデル選択 (3)
----





---
### ゴリゴリ君

「ゴリゴリ君」のデータで，検証データを用いてモデルやハイパーパラメータの値を選択する実験をやってみよう．


まずはいつものように準備から．



In [ ]:
# 準備あれこれ
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn
seaborn.set()

# scikit-learn のいろいろ
from sklearn.model_selection import KFold
from sklearn.neural_network import MLPClassifier

# データを読み込む
dfGori = pd.read_csv('https://www-tlab.math.ryukoku.ac.jp/~takataka/course/ML/gorigori.csv', header=0)

これまでは，気温やアイス売上数の値をそのまま使っていましたが，今回は正規化して実験します．ただし，適当な数で割るだけという簡易的な方法です．

In [ ]:
# データを正規化
dfGori['気温'] /= 40
dfGori['アイス売上数'] /= 150

以下のセルは，実験用の関数の定義です．

In [ ]:
## 1, x, x^2, x^3, ..., X^D をならべたデータ行列（N x (D+1)）をつくる
#
def makeDataMatrix(x, D):

    N = x.shape[0]
    X = np.zeros((N, D+1))
    X[:, 0] = 1
    for i in range(1, D+1):
        X[:, i] = x**i

    return X

##  正規方程式を解く（正則化付き）
#
def solve(X, y, alpha=0.0):

    N = X.shape[0]
    A = np.dot(X.T, X)
    b = np.dot(X.T, y)
    if alpha > 0.0:
        Dp1 = X.shape[1]
        Id = np.eye(Dp1)
        Id[0, 0] = 0.0
        A += alpha * Id

    # x is the solution of the equation Ax = b
    x = np.linalg.solve(A, b)

    return x

以下で実験を行います．ここでは，次のような条件で多項式当てはめのモデルを学習させます．

- 多項式の次数: $D = 10$ と $D = 20$ の2通り
- 正則化項の係数: $\alpha = 0, 0.001, 0.01, 0.1$ の4通り

それぞれを組み合わせた8通りの条件で学習を行い，汎化性能が最も高いと予想される条件を探します．
それぞれの条件で$n$-分割交差検証を行い，$n$通り得られる検証誤差の平均値を汎化性能の指標とします．$n$ は以下のコード中の変数 `nfold` で指定しています．
$n$-分割交差検証のためにデータを分割する処理には，[sklearn.model_selection.KFold](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html) を利用することにします．

In [ ]:
# データセットをいくつの組に分割するか
nfold = 5

# n-分割交差検証のための KFold クラスのインスタンスを生成
kf = KFold(n_splits=nfold, shuffle=True, random_state=2929)

msqeL = {}
msqeV = {}

# 多項式の次数 D
for D in [10, 20]:

    # 学習用と検証用のデータを準備
    Xorg = makeDataMatrix(dfGori['気温'].to_numpy(), D)
    Yorg = dfGori['アイス売上数'].to_numpy()

    # 正則化項の係数 alpha
    for alpha in [0.0, 0.001, 0.01, 0.1]:

        msqeLsub = np.empty(nfold)
        msqeVsub = np.empty(nfold)

        # nfold 組のデータのうち一つで学習と検証
        for i, (idxL, idxV) in enumerate(kf.split(Xorg)):
            # データを学習用と検証用に分ける
            XL, YL = Xorg[idxL], Yorg[idxL]
            XV, YV = Xorg[idxV], Yorg[idxV]
            # 正則化付き最小二乗法でD次多項式を当てはめ
            w = solve(XL, YL, alpha=alpha)
            # 学習データと検証データに対する平均二乗誤差を計算
            msqeLsub[i] = np.mean(np.square(YL - XL@w))
            msqeVsub[i] = np.mean(np.square(YV - XV@w))

        # nfold 回の学習で得られた平均二乗誤差の平均を記録
        msqeL[(D, alpha)] = np.mean(msqeLsub)
        msqeV[(D, alpha)] = np.mean(msqeVsub)


上記では，8通りの条件のそれぞれについて，5通りの学習-検証データの組を使って学習と検証を行っています．
以下のセルを実行すると，それぞれの条件について，5-分割交差検証によって得られた，学習データに対する平均二乗誤差の平均（`msqeL`）と，検証データに対する平均二乗誤差の平均（`msqeV`）が表示されます．

In [ ]:
for key in msqeL.keys():
    D, alpha = key
    print(f'D = {D} alpha = {alpha}   msqeL = {msqeL[key]:.3e}   msqeV = {msqeV[key]:.3e}')

上記の結果を見て，これら8通りの条件のうち，汎化性能が最も高そうなものを選んで，その条件を紙にメモしておきましょう．

また，その条件を以下のセルの `D = ???` と `alpha = ???` のところに指定してセルを実行してみましょう．
このセルでは，指定した条件で，全ての学習データを用いて再度学習を行い，その結果を表示します．

In [ ]:
# すべてのデータを使って多項式当てはめ
D = 20
alpha = 0.0
Xorg = makeDataMatrix(dfGori['気温'].to_numpy(), D)
Yorg = dfGori['アイス売上数'].to_numpy()
w = solve(Xorg, Yorg, alpha=alpha)

# 曲線の式の値を計算
Xr =  np.linspace(0, 1, 100)
XXr = makeDataMatrix(Xr, D)
Yest = XXr @ w
print(Yest.shape)

# グラフを描く
fig, ax = plt.subplots(1, facecolor='white', figsize=(8, 6))
ax.scatter(Xorg[:, 1], Yorg)
ax.plot(Xr, Yest, color='red', label=f'D = {D}, $\\alpha$ = {alpha}')
ax.set_xlim(0.0, 1)
ax.set_ylim(0.0, 1)
ax.legend()
plt.show()

---
### 手書き数字画像の識別

何度も使っている手書き数字画像の問題に階層型ニューラルネットワークを適用して，モデルを選択してみましょう．

In [ ]:
# 手書き数字データの入手
! wget -nc https://www-tlab.math.ryukoku.ac.jp/~takataka/course/ML/minimnist.npz
minimnist = np.load('minimnist.npz')
XLorg = minimnist['datL'].astype(float)
YLorg = minimnist['labL']
XT = minimnist['datT']
YT = minimnist['labT']

K = 10 # クラス数
D = XLorg.shape[1] # データの次元数 28 x 28 = 784

以下で実験を行います．ここでは，次の6通りのニューラルネットの中から，汎化性能が最も高いと予想されるものを選ぶことにします．

- ニューロン数 100-10 の2層ニューラルネット（中間層のニューロン数が1000で出力層のニューロン数が10）
- ニューロン数 500-10 の2層ニューラルネット
- ニューロン数 1000-10 の2層ニューラルネット
- ニューロン数 100-100-10 の3層ニューラルネット
- ニューロン数 500-100-10 の3層ニューラルネット（入力に近い方の中間層のニューロン数が500，出力に近い方の中間層のニューロン数が100）
- ニューロン数 1000-100-10 の3層ニューラルネット

それぞれのニューラルネットの学習の際には$n$-分割交差検証を行い，$n$通り得られる検証データの正答率の平均値を汎化性能の指標とします．$n$ は以下のコード中の変数 `nfold` で指定しています．

In [ ]:
# データセットをいくつの組に分割するか
nfold = 3

# n-分割交差検証のための KFold クラスのインスタンスを生成
kf = KFold(n_splits=nfold, shuffle=True, random_state=2929)

scoreL = {}
scoreV = {}

# ニューロン数
HL_list = [(100, ), (500, ), (1000,), (100, 100), (500, 100), (1000, 100)]

for HL in HL_list:

    print('ニューロン数', end=' ')
    if len(HL) == 1:
        print(f'{HL[0]}-{K}')
    elif len(HL) == 2:
        print(f'{HL[0]}-{HL[1]}-{K}')

    scoreLsub = np.empty(nfold)
    scoreVsub = np.empty(nfold)

    # nfold 組のデータのうち一つで学習と検証
    for i, (idxL, idxV) in enumerate(kf.split(XLorg)):

        print(f'{i+1}/{nfold}')

        # データを学習用と検証用に分ける
        XL, YL = XLorg[idxL], YLorg[idxL]
        XV, YV = XLorg[idxV], YLorg[idxV]
        # ニューラルネットの学習
        model = MLPClassifier(hidden_layer_sizes=HL)
        model.fit(XL, YL)
        # 学習データと検証データに対する識別率を算出
        scoreLsub[i] = model.score(XL, YL)
        scoreVsub[i] = model.score(XV, YV)

    # nfold 回の学習で得られた平均二乗誤差の平均を記録
    scoreL[HL] = np.mean(scoreLsub)
    scoreV[HL] = np.mean(scoreVsub)

次のセルを実行すると，それぞれのニューラルネットについて，3通りの学習データに対する正答率の平均（`scoreL`）と，3通りの検証データに対する正答率の平均（`scoreV`）が表示されます．

In [ ]:
for HL in scoreL.keys():
    print('ニューロン数', end=' ')
    if len(HL) == 1:
        print(f'{HL[0]}-{K}', end=' ')
    elif len(HL) == 2:
        print(f'{HL[0]}-{HL[1]}-{K}', end=' ')
    print(f'  scoreL = {scoreL[HL]:.3f}   scoreV = {scoreV[HL]:.3f}')

上記の結果を見て，これらの中で最も汎化性能が最も高そうなモデルを選んで，その構造（層の数やニューロン数）を紙にメモしておきましょう．

また，次のセルの `hidden_layer_sizes=(10,)` のところを適切に修正して実行すると，全ての学習データを使ってもう一度学習を行い，テストデータに対する正答率を算出します．選んだニューラルネットでやってみましょう．

In [ ]:
model = MLPClassifier(hidden_layer_sizes=(10,))
model.fit(XLorg, YLorg)
scoreT = model.score(XT, YT)
print(f'scoreT = {scoreT:.3f}')